# 알고리즘 : 랜덤 포레스트 (Random Forest)
### 미션 : 결정 트리의 발전된 형태인 기초인 랜덤 포레스트를 학습시켜 중고차 가격을 예측합니다. 
### 종속 변수 : selling_price(판매가격)
### 평가 지표 : RMSE
### 문제 유형 : 회귀

지도학습 알고리즘 6
랜덤 포레스트 (Random Forest) : 결정트리의 단점인 오버피팅 문제를 완화. 무수히 많은 랜덤 트리를 이용해 예측. 여러 모델(여기서는 결정트리)을 활용하여 하나의 모델을 이루는 기법을 앙상블 이라고 한다
*앙상블 : 여러 모델을 만들고 각 예측값들을 투표/평균 등으로 통합하여 더 정확한 예측을 도모하는 방법
앙상블 기법을 사용 트리 기반 모델 중 가장 보편적이며, 부스팅 모델에 비하면 예측력이나 속도에서 부족한 부분이 있고, 시각화에는 결정트리에 못미치나, 다음 단계인 부스팅 모델을 이해하려면 꼭 알아야하는 필수 알고리즘이다. 


학습 순서
1. 문제정의 -> 2. 라이브러리 및 데이터 불러오기/ 데이터 확인하기 -> 3. 전처리: a. 텍스트 데이터 b. 결측치 처리와 더미 변수 변환 -> 4. 데이터 모델링 및 예측 -> 5. 이해하기 a. K겹 교차검증 b. 랜덤 포레스트 -> 6. 하이퍼파라미터 튜닝하기 

정의 : 랜덤으로 독립적인 트리를 여러개 만들어서 결정 트리의 오버피팅 문제를 완화해 분류하는 알고리즘이다.  

장점 : 결정 트리와 마찬가지로, 아웃라이어에 거의 영향을 받지 않는다. 선형/비선형 데이터에 상관없이 잘 작동한다.  
단점 : 학습속도가 상대적으로 느리고, 수많은 트리를 동원하기 때문에 모델에 대한 해석이 어렵다. 

유용한 곳 : 종속변수가 연속형 데이터와 범주형 데이터인 경우 모두에서 사용, 아웃라이어가 문제되는 경우 선형 모델보다 좋은 대안, 오버피팅 문제로 결정 트리 어려울때 랜덤포레스트 사용 가능

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

file_url = 'https://media.githubusercontent.com/media/musthave-ML10/data_source/main/car.csv'
data = pd.read_csv(file_url)

In [2]:
data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0


In [4]:
data.info()
# 몇몇 값은 숫자로 되어 있어야되는데 object로 되어 있음. 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           8128 non-null   object 
 1   year           8128 non-null   int64  
 2   selling_price  8128 non-null   int64  
 3   km_driven      8128 non-null   int64  
 4   fuel           8128 non-null   object 
 5   seller_type    8128 non-null   object 
 6   transmission   8128 non-null   object 
 7   owner          8128 non-null   object 
 8   mileage        7907 non-null   object 
 9   engine         7907 non-null   object 
 10  max_power      7913 non-null   object 
 11  torque         7906 non-null   object 
 12  seats          7907 non-null   float64
dtypes: float64(1), int64(3), object(9)
memory usage: 825.6+ KB


In [5]:
round(data.describe(),2)
#가격에 아웃라이어가 존재하지만, 트리모델 사용할 것이니 별도 처리 않해도 됨
# 선형 모델은 아웃라이어 처리 필요

,year,selling_price,km_driven,seats
count,8128.00,8128.00,8128.00,7907.00
mean,2013.80,638271.81,69819.51,5.42
std,4.04,806253.40,56550.55,0.96
min,1983.00,29999.00,1.00,2.00
25%,2011.00,254999.00,35000.00,5.00
50%,2015.00,450000.00,60000.00,5.00
75%,2017.00,675000.00,98000.00,5.00
max,2020.00,10000000.00,2360457.00,14.00


9.3 전처리 : 텍스트 데이터

engine 변수 전처리하기

In [6]:
data['engine'].str.split(expand=True)
#str.split() : 판다스제공 함수, 문자형 데이터를 분리하는 함수. 
# 기본적으로 빈칸 기준으로 분리/ 콤마, 마침표 같은 특정문자를 기준으로 분리도 가능. 
# data['engine'].str.split("!")

,0,1
0,1248,CC
1,1498,CC
2,1497,CC
3,1396,CC
4,1298,CC
...,...,...
8123,1197,CC
8124,1493,CC
8125,1248,CC
8126,1396,CC


In [7]:
data[['engine','engine_unit']]= data['engine'].str.split(expand=True)

In [8]:
data['engine'].head()

0    1248
1    1498
2    1497
3    1396
4    1298
Name: engine, dtype: object

In [9]:
data['engine_unit'].head()

0    CC
1    CC
2    CC
3    CC
4    CC
Name: engine_unit, dtype: object

In [11]:
#enigne 컬럼 float로 변환
data['engine'] = data['engine'].astype('float32')

In [12]:
data['engine'].head()

0    1248.0
1    1498.0
2    1497.0
3    1396.0
4    1298.0
Name: engine, dtype: float32

In [13]:
#engine_unit의 고윳값 확인
data['engine_unit'].unique()

array(['CC', nan], dtype=object)

In [14]:
#cc는 필요 없으니 삭제
data.drop('engine_unit', axis=1, inplace= True)


max_power 변수 전처리하기

In [15]:
data[['max_power','max_power_unit']] = data['max_power'].str.split(expand=True)

In [16]:
data['max_power'].head()

0        74
1    103.52
2        78
3        90
4      88.2
Name: max_power, dtype: object

In [17]:
data['max_power'] = data['max_power'].astype('float32')

ValueError: could not convert string to float: 'bhp'

ValueError: could not convert string to float: 'bhp'

--> 'bhp' 라는 string을 float로 바꿀수 없다, 
데이터 값 확인 필요

In [18]:
data[data['max_power'] == 'bhp']
# 보통 한개밖에 없으면 삭제해도 되지만,,,,
# 잘못된 데이터가 여러건 있을 때는 "Try and Except"블록을 사용 해서 처리

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,max_power_unit
4933,Maruti Omni CNG,2000,80000,100000,CNG,Individual,Manual,Second Owner,10.9 km/kg,796.0,bhp,NaN,8.0,None


In [19]:
def isFloat(value):
  try: # 시도
    float(value) #값을 숫자로 변환
    return float(value) #변환된 값 리턴
  except ValueError: # try 에서 ValueError난 경우
    return np.NaN #np.NaN 리턴

In [20]:
data['max_power'] = data['max_power'].apply(isFloat)
#isFloat 사용하여 숫자형 변수로 변환

In [21]:
data['max_power_unit'].unique()
#bhp말고도 Null값도 있네.

array(['bhp', nan, None], dtype=object)

In [22]:
data.drop('max_power_unit', axis=1, inplace=True)

mileage 변수 전처리 하기


In [23]:
data[['mileage','mileage_unit']] = data['mileage'].str.split(expand=True)

In [24]:
data['mileage'] = data['mileage'].astype('float32')

In [25]:
data['mileage_unit'].unique()
#LPG나 CNG는 킬로그램 단위

array(['kmpl', 'km/kg', nan], dtype=object)

In [26]:
data['fuel'].unique()

array(['Diesel', 'Petrol', 'LPG', 'CNG'], dtype=object)

#### 구글 찾아보면 현재 위 원자재 상품 가격 나옴
- mileage변수를 각 연료별 가격으로 나누면 1달러당 주형거리가 된다

In [28]:
def mile(x):
    if x['fuel'] == 'Petrol':
        return x['mileage'] / 80.43
    elif x['fuel'] == 'Diesel':
        return x['mileage'] / 73.56
    elif x['fuel'] == 'LPG':
        return x['mileage'] / 40.85
    else: 
        return x['mileage'] / 44.23

In [29]:
#mileage컬럼을 새로운 mileage컬럼으로 수정 (달러당 마일로)
data['mileage'] = data.apply(mile, axis=1)

In [30]:
data.drop('mileage_unit',axis=1,inplace=True)

torque 변수 처리
1. 단위 앞부분 숫자만 출력해서 숫자형으로 바꾼 후 
2. Nm단위로 스케일링

In [31]:
data['torque'].head()

0              190Nm@ 2000rpm
1         250Nm@ 1500-2500rpm
2       12.7@ 2,700(kgm@ rpm)
3    22.4 kgm at 1750-2750rpm
4       11.5@ 4,500(kgm@ rpm)
Name: torque, dtype: object

In [32]:
#대소문자 섞여 있으니까 우선 다 대문자로 
data['torque'] = data['torque'].str.upper()

In [33]:
def torque_unit(x):
    if 'NM' in str(x):
        return 'Nm'
    elif 'KGM' in str(x):
        return 'kgm'

In [37]:
data['torque_unit'] = data['torque'].apply(torque_unit)
#torque 변수를 torque_unit 함수에 적용하여 torque_unit이라는 변수로 저장

In [40]:
data['torque_unit'].unique()

array(['Nm', 'kgm', None], dtype=object)

In [41]:
#data['torque_unit'].isna() # torque_unit이 Null값인지 확인
#data[data['torque_unit'].isna()] #얻은 결과를 data[] 안에 넣어서 필터링
data[data['torque_unit'].isna()]['torque'].unique() #위에 두개를 한문장으로
#torqe_unit이 결측치인 라인의 torque변수 고윳값 확인

array([nan, '250@ 1250-5000RPM', '510@ 1600-2400', '110(11.2)@ 4800',
       '210 / 1900'], dtype=object)

In [42]:
#결과를 보니 Nm, kgm도 없는 데이터랑 NAN데이터도 있음. 
# 둘중 하나라는 전재하에, 백단위는 보통 Nm /// 10, 20단위는 kgm으로 추측

data['torque_unit'].fillna('Nm', inplace= True)

In [43]:
string_example = '12.7@ 2,700(KGM@ RPM)'

In [44]:
string_example[:4]

'12.7'

In [49]:
# enumerate()
# i는 0부터 시작하여 for문이 끝날 때 까지 반복되는 숫자만큼 올라가며, j는 해당 위치의 값을 보여준다.
for i, j in enumerate(string_example):
    print(i, '번째 텍스트: ', j)

0 번째 텍스트:  1
1 번째 텍스트:  2
2 번째 텍스트:  .
3 번째 텍스트:  7
4 번째 텍스트:  @
5 번째 텍스트:   
6 번째 텍스트:  2
7 번째 텍스트:  ,
8 번째 텍스트:  7
9 번째 텍스트:  0
10 번째 텍스트:  0
11 번째 텍스트:  (
12 번째 텍스트:  K
13 번째 텍스트:  G
14 번째 텍스트:  M
15 번째 텍스트:  @
16 번째 텍스트:   
17 번째 텍스트:  R
18 번째 텍스트:  P
19 번째 텍스트:  M
20 번째 텍스트:  )


In [47]:
for i,j in enumerate(string_example): 
    if j not in '0123456789.': #만약 j가 01234356789.에 포함되지 않으면
        cut = i #인덱스(순서)를 cut에 저장
        break

In [ ]:
def split_num(x):
    x = str(x)  # 문자형태로 전환
    for i,j in enumerate(x): # 인덱스를 포함한 순회
        if j not in '0123456789.': # j가 0123456789. 에 속하지 않으면
            cut = i # 인덱스를 cut에 저장
            break
    return x[:cut] # 